In [15]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import defaultCityDistricts

import classroom
import teachers
import course
import aClass
import students
import meeting
import exams
import surveys
import attendance


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
df_classrooms = pd.read_csv('dataFullSnapshot1/classrooms.csv')
# df_classrooms = classroom.generate(357, 999)

In [17]:
df_teachers = pd.read_csv('dataFullSnapshot1/teachers.csv')
# df_teachers = teachers.generate(300)

In [18]:
df_courses = pd.read_csv('dataFullSnapshot1/courses.csv')
# df_courses = course.generate(300, df_teachers)

In [19]:
df_classes = pd.read_csv('dataFullSnapshot1/classes.csv')
# df_classes, df_courses = aClass.generate(10, df_teachers, df_courses)

In [20]:
df_students = pd.read_csv('dataFullSnapshot1/students.csv')
# df_students = students.generate(800, df_classes)

In [21]:
# may enter infinite loop; if it happens restart the kernel and run the code again
df_meetings = pd.read_csv('data/meetings.csv')
# df_meetings = meeting.generate(0, df_courses, df_classrooms, df_classes, "2017-09-01", "2022-07-01")

In [22]:
df_exams = pd.read_csv('data/exams.csv')
# df_exams = exams.generate(0, df_meetings, df_students)

In [23]:
df_surveys = pd.read_csv('data/surveys.csv')
# df_surveys = surveys.generate(0, df_meetings, df_students, df_courses)

In [24]:
df_attendance = pd.read_csv('data/attendance.csv')
# df_attendance = attendance.generate(0, df_meetings, df_students, df_teachers, df_courses)

In [ ]:
#purging unused rows
size =len(df_classrooms)
df_classrooms = df_classrooms[df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
df_classrooms_removed = df_classrooms[~df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
print(f"Removed {size - len(df_classrooms)} classrooms")

size =len(df_teachers)
df_teachers = df_teachers[df_teachers["tID"].isin(pd.concat([df_classes["tID"], df_courses["tID"]]).unique())]
df_teachers_removed = df_teachers[~df_teachers["tID"].isin(df_classes["tID"] + df_courses["tID"])]
print(f"Removed {size - len(df_teachers)} teachers")

size =len(df_courses)
df_courses = df_courses[df_courses["cID"].isin(df_classes["courses"].explode().unique())]
df_courses_removed = df_courses[~df_courses["cID"].isin(df_classes["courses"].explode().unique())]
print(f"Removed {size - len(df_courses)} courses")

size =len(df_classes)
df_classes = df_classes[df_classes["ClassName"].isin(df_students["ClassName"])]
df_courses_classes = df_classes[~df_classes["ClassName"].isin(df_students["ClassName"])]
print(f"Removed {size - len(df_classes)} classes")

In [ ]:
headers = True
path = "dataFullSnapshot2/"
df_students.drop(columns=["iq"]).to_csv(f"{path}students.csv", index=False, header=headers)
df_classes.drop(columns=["Year", "courses"]).to_csv(f"{path}classes.csv", index=False, header=headers)
df_teachers.to_csv(f"{path}teachers.csv", index=False, header=headers)
df_courses.to_csv(f"{path}courses.csv", index=False, header=headers)
df_classrooms.to_csv(f"{path}classrooms.csv", index=False, header=headers)
df_meetings.drop(columns=["StartHour", "EndHour", "Year", "Month"]).to_csv(f"{path}meetings.csv", index=False, header=headers)
df_surveys.to_csv(f"{path}surveys.csv", index=False, header=headers)
df_exams.to_csv(f"{path}exams.csv", index=False, header=headers)
df_attendance.to_csv(f"{path}attendance.csv", index=True, header=headers)


In [27]:
df = df_teachers.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Teachers SET CityDistrict = '{x['CityDistrict2']}' WHERE tID = {x['tID']};", axis=1)
for x in df["update"]:
    print(x)

Update Teachers SET CityDistrict = 'Reda' WHERE tID = 289;
Update Teachers SET CityDistrict = 'Redlowo' WHERE tID = 1;
Update Teachers SET CityDistrict = 'Gdansk centrum' WHERE tID = 5;
Update Teachers SET CityDistrict = 'Gdynia centrum' WHERE tID = 50;
Update Teachers SET CityDistrict = 'Reda' WHERE tID = 19;
Update Teachers SET CityDistrict = 'Jasien' WHERE tID = 222;
Update Teachers SET CityDistrict = 'Lostowice' WHERE tID = 52;
Update Teachers SET CityDistrict = 'Gdansk centrum' WHERE tID = 210;
Update Teachers SET CityDistrict = 'Wejherowo' WHERE tID = 122;
Update Teachers SET CityDistrict = 'Zaspa' WHERE tID = 271;
Update Teachers SET CityDistrict = 'Zaspa' WHERE tID = 64;
Update Teachers SET CityDistrict = 'Stocznia' WHERE tID = 117;
Update Teachers SET CityDistrict = 'Lostowice' WHERE tID = 215;
Update Teachers SET CityDistrict = 'Zaspa' WHERE tID = 296;
Update Teachers SET CityDistrict = 'Orunia' WHERE tID = 187;


In [82]:
df = df_students.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET CityDistrict = '{x['CityDistrict2']}' WHERE sID = {x['sID']};", axis=1)
for x in df["update"]:
    print(x)

Update Students SET CityDistrict = 'Przymorze' WHERE sID = 77;
Update Students SET CityDistrict = 'Gdansk centrum' WHERE sID = 31;
Update Students SET CityDistrict = 'Gdansk centrum' WHERE sID = 374;
Update Students SET CityDistrict = 'Wejherowo' WHERE sID = 676;
Update Students SET CityDistrict = 'Wejherowo' WHERE sID = 568;
Update Students SET CityDistrict = 'Przymorze' WHERE sID = 688;
Update Students SET CityDistrict = 'Przymorze' WHERE sID = 657;
Update Students SET CityDistrict = 'Zaspa' WHERE sID = 710;
Update Students SET CityDistrict = 'Lostowice' WHERE sID = 659;
Update Students SET CityDistrict = 'Przymorze' WHERE sID = 76;
Update Students SET CityDistrict = 'Zaspa' WHERE sID = 101;
Update Students SET CityDistrict = 'Przymorze' WHERE sID = 433;
Update Students SET CityDistrict = 'Wrzeszcz' WHERE sID = 491;
Update Students SET CityDistrict = 'Wejherowo' WHERE sID = 502;
Update Students SET CityDistrict = 'Przymorze' WHERE sID = 119;


In [83]:
df = df_students.sample(n=15)
df["ClassName2"] = df["ClassName"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET ClassName = '{x['ClassName']}' WHERE sID = {x['sID']};", axis=1)
for x in df["update"]:
    print(x)

Update Students SET ClassName = 'E20' WHERE sID = 28;
Update Students SET ClassName = 'B20' WHERE sID = 478;
Update Students SET ClassName = 'J19' WHERE sID = 403;
Update Students SET ClassName = 'E17' WHERE sID = 516;
Update Students SET ClassName = 'I22' WHERE sID = 500;
Update Students SET ClassName = 'A19' WHERE sID = 142;
Update Students SET ClassName = 'B22' WHERE sID = 244;
Update Students SET ClassName = 'B21' WHERE sID = 680;
Update Students SET ClassName = 'A17' WHERE sID = 526;
Update Students SET ClassName = 'F21' WHERE sID = 375;
Update Students SET ClassName = 'C22' WHERE sID = 7;
Update Students SET ClassName = 'J22' WHERE sID = 418;
Update Students SET ClassName = 'E22' WHERE sID = 101;
Update Students SET ClassName = 'F17' WHERE sID = 9;
Update Students SET ClassName = 'H20' WHERE sID = 211;


In [26]:
df = df_classes.sample(n=15)
df["tID2"] = df["tID"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Class SET tID = '{x['tID']}' WHERE ClassName = '{x['ClassName']}';", axis=1)
for x in df["update"]:
    print(x)

Update Class SET tID = '33' WHERE ClassName = 'B22';
Update Class SET tID = '25' WHERE ClassName = 'J22';
Update Class SET tID = '87' WHERE ClassName = 'G18';
Update Class SET tID = '48' WHERE ClassName = 'H18';
Update Class SET tID = '60' WHERE ClassName = 'H19';
Update Class SET tID = '112' WHERE ClassName = 'D18';
Update Class SET tID = '276' WHERE ClassName = 'A21';
Update Class SET tID = '296' WHERE ClassName = 'D22';
Update Class SET tID = '188' WHERE ClassName = 'C18';
Update Class SET tID = '50' WHERE ClassName = 'J21';
Update Class SET tID = '274' WHERE ClassName = 'J20';
Update Class SET tID = '121' WHERE ClassName = 'B20';
Update Class SET tID = '108' WHERE ClassName = 'I20';
Update Class SET tID = '269' WHERE ClassName = 'J19';
Update Class SET tID = '126' WHERE ClassName = 'G19';


In [29]:
df_meetings

,mID,Date,RoomNumber,cID,ClassName
0,0,2021-09-01 09:00:00,46,71,A22
1,1,2021-09-01 09:00:00,46,116,A22
2,2,2021-09-01 09:00:00,46,166,A22
3,3,2021-09-01 09:00:00,46,232,A22
4,4,2021-09-01 09:00:00,46,233,A22
5,5,2021-09-01 09:00:00,46,265,A22
6,6,2021-09-01 09:00:00,46,283,A22
7,7,2021-09-01 09:00:00,46,300,A22
8,8,2021-09-01 17:00:00,50,6,A21


In [37]:
df = pd.read_csv('dataFullSnapshot2/surveys.csv')
df["mID"] = df["mID"] + 487350
df.to_csv('dataFullSnapshot2/surveys2.csv',  index=False, header=False)

,mID,Date,RoomNumber,cID,ClassName
0,487349,2017-09-01 09:00:00,118,17,A22
1,487350,2017-09-01 09:00:00,118,76,A22
2,487351,2017-09-01 09:00:00,118,97,A22
3,487352,2017-09-01 09:00:00,118,157,A22
4,487353,2017-09-01 09:00:00,118,212,A22
...,...,...,...,...,...
486895,974244,2022-06-30 19:00:00,456,210,J17
486896,974245,2022-06-30 13:00:00,24,241,J17
486897,974246,2022-06-30 13:00:00,24,243,J17
486898,974247,2022-06-30 14:00:00,383,271,J17
